# Collect All Functions
------------------
This project comes with functions that collect all STIX objects from all ATT&CK Matrices at once. These functions help collect more with less API call requests.

## Import ATTACK API Client

In [1]:
from attackcti import attack_client

## Import Extra Libraries

In [8]:
import pandas
import json

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)

## Initialize ATT&CK Client Variable

In [4]:
lift = attack_client()

## Get All Techniques
We can extract all STIX objects of type `attack-pattern` (technique) across all ATT&CK matrices.
* By default, this function removes `deprecated` and `revoked` techniques. If you want to keep those techniques in the results, you can run the function with the parameter `skip_revoked_deprecated=False`.
* By default, this function also includes all techniques and sub-techniques. If you want to only get techniques that are not sub-techniques, you can run the function with the parameter `include_subtechniques=False`.

In [5]:
techniques = lift.get_techniques()

In [6]:
print("Number of Techniques in ATT&CK")
len(techniques)

Number of Techniques in ATT&CK


839

By default, the data returned by the available functions in the attackcti library is of type **stix2**. However, if you want to interact with libraries such as **Pandas**, it needs to be of type **dict**

In [9]:
all_techniques = []
for t in techniques:
    all_techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(all_techniques)
df.reindex(['created','name', 'x_mitre_data_sources', 'x_mitre_platforms'], axis=1)[0:5]

,created,name,x_mitre_data_sources,x_mitre_platforms
0,2024-03-29T18:07:04.743Z,AutoHotKey & AutoIT,"[Process: Process Creation, Command: Command E...",[Windows]
1,2024-03-29T16:59:10.374Z,File/Path Exclusions,[File: File Creation],"[Linux, macOS, Windows]"
2,2024-03-29T12:38:17.135Z,Encrypted/Encoded File,"[File: File Creation, File: File Metadata]","[Linux, macOS, Windows]"
3,2024-03-28T15:36:34.141Z,AppDomainManager,"[Module: Module Load, File: File Creation, Pro...",[Windows]
4,2024-03-28T03:29:35.616Z,Network Devices,[Internet Scan: Response Content],[PRE]


We can now access the schema of the dataframe

In [10]:
list(df)

['type',
 'id',
 'created_by_ref',
 'created',
 'modified',
 'name',
 'description',
 'kill_chain_phases',
 'external_references',
 'object_marking_refs',
 'x_mitre_attack_spec_version',
 'x_mitre_contributors',
 'x_mitre_data_sources',
 'x_mitre_deprecated',
 'x_mitre_detection',
 'x_mitre_domains',
 'x_mitre_is_subtechnique',
 'x_mitre_modified_by_ref',
 'x_mitre_platforms',
 'x_mitre_remote_support',
 'x_mitre_version',
 'x_mitre_impact_type',
 'x_mitre_system_requirements',
 'x_mitre_effective_permissions',
 'x_mitre_permissions_required',
 'x_mitre_defense_bypassed',
 'x_mitre_network_requirements',
 'x_mitre_tactic_type']

Showing one technique example:

In [11]:
techniques[0]

AttackPattern(type='attack-pattern', id='attack-pattern--3a32740a-11b0-4bcf-b0a9-3abd0f6d3cd5', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2024-03-29T18:07:04.743Z', modified='2024-04-10T16:05:22.456Z', name='AutoHotKey & AutoIT', description="Adversaries may execute commands and perform malicious tasks using AutoIT and AutoHotKey automation scripts. AutoIT and AutoHotkey (AHK) are scripting languages that enable users to automate Windows tasks. These automation scripts can be used to perform a wide variety of actions, such as clicking on buttons, entering text, and opening and closing programs.(Citation: AutoIT)(Citation: AutoHotKey)\n\nAdversaries may use AHK (`.ahk`) and AutoIT (`.au3`) scripts to execute malicious code on a victim's system. For example, adversaries have used for AHK to execute payloads and other modular malware such as keyloggers. Adversaries have also used custom AHK files containing embedded malware as [Phishing](https://attack.mitr

### Enrich Techniques Data Sources
As you might already know, the ATT&CK data model now represents `data sources` as objects. However, when retrieving techniques from ATT&CK TAXII server, their data sources section only includes data sources and data components names. Therefore, we created a parameter that you can use with the `get_techniques()` function to enrich the data sources section of each technique. The parameter `enrich_data_sources` is set to `False` by default.

In [12]:
techniques = lift.get_techniques(enrich_data_sources=True)

This function returns a list of techniques with the `x_mitre_data_sources` attribute as a list of STIX objects representing data sources with their respective data components depending on the technique's detection context.

In [13]:
for ds in techniques[0]['x_mitre_data_sources']:
    for dc in ds['data_components']:
        print(ds['name'], '-', dc['name'])

Command - Command Execution
Process - Process Creation


In [14]:
techniques[0]['x_mitre_data_sources'][0]

{'modified': '2023-04-20T18:38:00.625Z',
 'name': 'Command',
 'description': 'A directive given to a computer program, acting as an interpreter of some kind, in order to perform a specific task(Citation: Confluence Linux Command  Line)(Citation: Audit OSX)',
 'type': 'x-mitre-data-source',
 'id': 'x-mitre-data-source--73691708-ffb5-4e29-906d-f485f6fa7089',
 'created': '2021-10-20T15:05:19.273Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'revoked': False,
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/datasources/DS0017',
   'external_id': 'DS0017'},
  {'source_name': 'Confluence Linux Command  Line',
   'description': 'Confluence Support. (2021, September 8). How to enable command line audit logging in linux. Retrieved September 23, 2021.',
   'url': 'https://confluence.atlassian.com/confkb/how-to-enable-command-line-audit-logging-in-linux-956166545.html'},
  {'source_name': 'Audit OSX',
   'description': 'Gaglia

## Get All Groups
We can also extract all the available groups across all ATT&CK matrices at once.

In [15]:
groups = lift.get_groups()

In [16]:
print("Number of Groups in ATT&CK")
len(groups)

Number of Groups in ATT&CK


152

In [17]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,type,id,created_by_ref,created,modified,name,description,aliases,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_contributors,x_mitre_deprecated,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_version
0,intrusion-set,intrusion-set--8332952e-b86b-486b-acc3-1c2a85d...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-03-26T18:38:00.759Z,2024-04-16T15:31:48.747Z,APT-C-23,[APT-C-23](https://attack.mitre.org/groups/G10...,"[APT-C-23, Mantis, Arid Viper, Desert Falcon, ...","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,[Sittikorn Sangrattanapitak],False,"[mobile-attack, enterprise-attack]",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0
1,intrusion-set,intrusion-set--bf668120-e9a6-4017-a014-bfc0f52...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-03-13T20:23:54.698Z,2024-03-29T14:10:35.711Z,Malteiro,[Malteiro](https://attack.mitre.org/groups/G10...,[Malteiro],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,"[Daniel Fernando Soriano Espinosa, SCILabs]",False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0
2,intrusion-set,intrusion-set--46bb06cb-f2d9-4b37-8c92-a27e224...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-02-20T23:59:25.966Z,2024-04-08T17:35:16.134Z,Akira,[Akira](https://attack.mitre.org/groups/G1024)...,"[Akira, GOLD SAHARA, PUNK SPIDER]","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,NaN,False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0
3,intrusion-set,intrusion-set--c1aab4c9-4c34-4f4f-8541-d529e46...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-02-05T19:27:35.655Z,2024-03-14T18:53:21.577Z,APT5,[APT5](https://attack.mitre.org/groups/G1023) ...,"[APT5, Mulberry Typhoon, MANGANESE, BRONZE FLE...","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,[@_montysecurity],False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0


## Get All Software
We can extract all Enterprise, Mobile and ICS (Software Malware & Tools)

In [18]:
software = lift.get_software()

In [19]:
print("Number of Software in ATT&CK")
len(software)

Number of Software in ATT&CK


794

In [21]:
software_list = []
for t in software:
    software_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(software_list)
df[0:4]

,type,id,created_by_ref,created,modified,name,description,labels,external_references,object_marking_refs,x_mitre_aliases,x_mitre_attack_spec_version,x_mitre_contributors,x_mitre_deprecated,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_platforms,x_mitre_version
0,tool,tool--1b3b8f96-43b1-4460-8e02-1f53d7802fb9,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-09-28T13:21:49.652Z,2023-10-19T12:18:43.123Z,Pacu,Pacu is an open-source AWS exploitation framew...,[tool],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[Pacu],3.2.0,"[Thanabodi Phrakhun, @naikordian]",False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[IaaS],1.0
1,tool,tool--6a5947f3-1a36-4653-8734-526df3e1d28d,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-09-20T17:32:59.932Z,2023-10-10T17:19:12.868Z,AsyncRAT,[AsyncRAT](https://attack.mitre.org/software/S...,[tool],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[AsyncRAT],3.2.0,[Aaron Jornet],False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],1.0
2,tool,tool--2f7f03bb-f367-4a5a-ad9b-310a12a48906,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-09-14T18:56:34.771Z,2023-09-25T18:54:49.773Z,ngrok,[ngrok](https://attack.mitre.org/software/S050...,[tool],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[ngrok],3.1.0,[Janantha Marasinghe],False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],1.2
3,tool,tool--e33267fe-099f-4af2-8730-63d49f8813b2,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-03-29T20:19:26.940Z,2023-08-03T18:30:05.885Z,Rubeus,[Rubeus](https://attack.mitre.org/software/S10...,[tool],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,[Rubeus],3.1.0,"[Mayuresh Dani, Qualys, Akshat Pradhan, Qualys]",False,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,[Windows],1.1


## Get All Relationships
We can also get all relationships from all ATT&CK matrices with one API request.

In [22]:
relationships = lift.get_relationships()

In [23]:
print("Number of Relationships in ATT&CK")
len(relationships)

Number of Relationships in ATT&CK


20868

In [24]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,type,id,created_by_ref,created,modified,relationship_type,description,source_ref,target_ref,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_deprecated,x_mitre_modified_by_ref,x_mitre_version
0,relationship,relationship--dbec2814-4a92-4ab7-a007-123b40b1...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-04-18T15:25:34.867Z,2024-04-18T15:31:12.348Z,uses,During [C0027](https://attack.mitre.org/campai...,intrusion-set--44d37b89-a739-4810-9111-0d2617a...,attack-pattern--6a5d222a-a7e0-4656-b110-782c33...,[{'source_name': 'MSTIC Octo Tempest Operation...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,0.1
1,relationship,relationship--f70697d6-11d6-4f1d-81ab-65bbe0d4...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-04-18T15:11:02.249Z,2024-04-18T18:48:56.566Z,uses,[Malteiro](https://attack.mitre.org/groups/G10...,intrusion-set--bf668120-e9a6-4017-a014-bfc0f52...,attack-pattern--c1b68a96-3c48-49ea-a6c0-9b2735...,"[{'source_name': 'SCILabs Malteiro 2021', 'des...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,0.1
2,relationship,relationship--722ffa83-29e2-48bb-a7c1-1cb52fb4...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-04-18T15:01:22.779Z,2024-04-18T15:01:22.779Z,uses,[Mispadu](https://attack.mitre.org/software/S1...,malware--4e6464d2-69df-4e56-8d4c-1973f84d7b80,attack-pattern--43e7dc91-05b2-474c-b9ac-2ed4fe...,[{'source_name': 'Segurança Informática URSA S...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,0.1
3,relationship,relationship--5b920225-ef1a-49f1-be65-1baa948d...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2024-04-18T14:14:59.504Z,2024-04-18T14:14:59.504Z,uses,[Malteiro](https://attack.mitre.org/groups/G10...,intrusion-set--bf668120-e9a6-4017-a014-bfc0f52...,attack-pattern--851e071f-208d-4c79-adc6-5974c8...,"[{'source_name': 'SCILabs Malteiro 2021', 'des...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,0.1


## Get All Data Sources
Now that `data sources` are part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [25]:
data_sources = lift.get_data_sources()

In [26]:
print("Number of Data Sources in ATT&CK")
len(data_sources)

Number of Data Sources in ATT&CK


42

In [27]:
data_sources[0]

{'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'id': 'x-mitre-data-source--38fe306c-bdec-4f3d-8521-b72dd32dbd17',
 'type': 'x-mitre-data-source',
 'created': '2021-10-20T15:05:19.275Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/datasources/DS0035',
   'external_id': 'DS0035'}],
 'modified': '2021-10-20T15:05:19.275Z',
 'name': 'Internet Scan',
 'description': 'Information obtained (commonly via active network traffic probes or web crawling) regarding various types of resources and servers connected to the public Internet',
 'x_mitre_platforms': ['PRE'],
 'x_mitre_domains': ['enterprise-attack'],
 'x_mitre_collection_layers': ['OSINT'],
 'x_mitre_version': '1.0',
 'x_mitre_attack_spec_version': '2.1.0',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5'}

## Get All Data Components
Now that `data components` are also part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [28]:
data_components = lift.get_data_components()

In [29]:
print("Number of data components in ATT&CK")
len(data_components)

Number of data components in ATT&CK


119

In [30]:
data_components[0]

{'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'id': 'x-mitre-data-component--1067aa74-5796-4d9b-b4f1-a4c9eb6fd9da',
 'type': 'x-mitre-data-component',
 'created': '2021-10-20T15:05:19.275Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'modified': '2021-10-20T15:05:19.275Z',
 'name': 'Response Metadata',
 'description': 'Contextual data about an Internet-facing resource gathered from a scan, such as running services or ports',
 'x_mitre_data_source_ref': 'x-mitre-data-source--38fe306c-bdec-4f3d-8521-b72dd32dbd17',
 'x_mitre_version': '1.0',
 'x_mitre_attack_spec_version': '2.1.0',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5'}